# Partie Thermique

---

In [39]:
from numpy import linspace, zeros
from numpy import linalg
from numpy import ndarray
import plotly.express as px

In [12]:
#Initialisation des constante

lbda = 0.96
rho = 2550
Cp = 840

N = 100
R = 1

t = 10
Nt = 10000

Text = linspace(1770,1670, Nt)
Tref = 1770

In [13]:
#Calcule des constantes
W = lbda/(rho * Cp)
dr = R / N
dt = t / Nt

T = []

#Initialisation de la temperaure
for i in range(Nt):
    T.append( ([Tref]*N)  + [Text[i]] )



def sq(x):
    return x*x

In [14]:

for m in range(1, Nt):

    T[m][0] = W * dt * (T[m-1][1] - T[m-1][0])/sq(dr) + T[m-1][0]

    for i in range(1,N):
        T[m][i] = W * dt * ( (T[m-1][i+1] - 2*T[m-1][i] + T[m-1][i-1])/sq(dr) + (4*(T[m-1][i+1] - T[m-1][i])) / ( sq(dr) * (2*i - 1))) + T[m-1][i]

In [15]:
#Tim = [ [ (T[m][i] + T[m][i-1])/2 for i in range(1, N+0)] for m in range(0, Nt)]
DT = [ [Tref - T[m][i] for i in range(len(T[0])-1)] for m in range(len(T))]

In [16]:
X = linspace(0, R, N)
Y = T[-1][:-1]

px.line(x = X , y = Y)

In [29]:
DT = []
X = linspace(0, R, N)
DT.append([ -(r*r/(R*R) * 100) for r in X] )

px.line(x = X, y = DT[0], title=r"Temperature en focntion de R")

# Partie Mécanique

---

In [18]:
p = 1

E = 71.5e9
nu = 0.21
kc = E/(3*(1-2*nu))
muc= E/(2*(1+nu))
alpha_c=9.3e-6

In [19]:
def k(i) -> float:
    return kc

def mu(i) -> float:
    return muc


def A(i) -> float:
    A = 4/(9*k(i)) + 1/(12 * mu(i))
    return A

def B(i) -> float:
    B = 1/(3*k(i))
    return B

def alpha(i) -> float:
    return alpha_c


## Construction de M et de R

In [20]:
M = [[0 for i in range(2*N)] for j in range(2*N)]
Rmat = [0]*(2*N)

M[0][0] = 1

M[2*N-1][2*N-2] = dr * N
M[2*N-1][2*N-1] = -1
Rmat[2*N-1] = p


temps = 0#Nt-1


for i in range(1, N):
    r = dr * i
    M[ 2*(i-1) + 1][ 2*(i-1) : 2*(i-1) + 4] = [-r, -1, r, 1]

for i in range(1,N):
    r = dr*i
    M[2*(i-1)+2][2*(i-1) : 2*(i-1) + 4] = [-A(i)*r, -B(i), A(i+1)*r, B(i+1)]

    Rmat[2*(i-1)+2] = alpha(i-1)*DT[temps][i-1]-alpha(i+1)*DT[temps][i]

In [44]:
def Tcontraintesi(sol_ab,i,r) -> ndarray:
    '''Calcule de la contrainte a partir des solution
    Attention i>0'''

    sigmai=zeros((3,3))
    ai,bi=sol_ab[2*i-2],sol_ab[2*i-1]
    sigmai[0][0]=ai*r+bi
    sigmai[1][1]=1.5*ai*r+bi
    sigmai[2][2]=1.5*ai*r+bi

    return sigmai

In [45]:
sol = linalg.solve(M, Rmat)

X = linspace(0, R, N)
Y1, Y2, Y3 = [], [], []

for i in range(1, N+1):
    r = i*dr
    contrainte = Tcontraintesi(sol, i, r)
    Y1.append(contrainte[0][0])
    Y2.append(contrainte[1][1])
    Y3.append(contrainte[2][2])



px.line(x = X, y = [Y1, Y2, Y3])
